In [1]:
import model as m
from sklearn.linear_model import LogisticRegression

In [2]:
# we intend to test each solver independently and then compare them afterwards
clf_newton = m.Classifier(1,LogisticRegression(solver ='newton-cg'))
clf_saga = m.Classifier(1,LogisticRegression(solver ='saga'))
clf_lib = m.Classifier(1,LogisticRegression(solver ='liblinear'))
clf_sag = m.Classifier(1,LogisticRegression(solver ='sag'))

In [3]:
#stop word sets specified for this data set
path = "../data/stop_words_news_groups/"
with open(path+"stop_words_no_numbers.txt", 'r') as content_file:
        content = content_file.read().replace(" ","").replace("\n","").split(",")
        stop_words_no_nums = frozenset(content)
with open(path+"stop_words_with_nums.txt", 'r') as content_file:
        content = content_file.read().replace(" ","").replace("\n","").split(",")
        stop_words_with_nums = frozenset(content)


In [6]:
params_trial_1 = { 
            #penalty is l2 by default
            'vect__max_features': (1000, 10000), # we dont want features to exceed observations (because of overfitting)
            'vect__ngram_range': ((1, 1), (1, 2)),
            'tfidf__norm': ('l1','l2',None),
            'vect__stop_words' : [stop_words_no_nums, stop_words_with_nums],
            'clf__max_iter': ([50,150]), #two extremes
            'clf__C':(10,1.0),
            }

In [ ]:
#### TEST 1 looking at saga ####
clf_saga.fit(params_trial_1,5)

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.8s


In [ ]:
clf_saga.eval_on_test([],False)
clf_saga.learning_curve([0.2,0.4,0.6,0.8,1.0])

In [ ]:
        from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
        from sklearn.feature_extraction.text import TfidfTransformer
        from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,plot_confusion_matrix
        from sklearn.model_selection import GridSearchCV,learning_curve
        import pandas as pd
        import numpy as np
        from pprint import pprint
        from time import time
        import matplotlib.pyplot as plt
        from sklearn.pipeline import Pipeline
        
        print("scores!")
        means = clf_saga.clf.cv_results_['mean_test_score']
        stds = clf_saga.clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf_saga.clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for"
                % (mean, std * 2))
            
        print("Best score:")
        print("%0.3f (+/-%0.03f)" % (clf_saga.clf.best_score_, std * 2))
        print("with parameters set:")
        best_parameters = clf_saga.clf.best_estimator_.get_params()
        for param_name in sorted(params_trial_1.keys()):
            print("\t%s: %r" % (param_name, best_parameters[param_name]))